In [1]:
import pandas as pd
import os
import glob


In [2]:
os.makedirs('Data/final_data',exist_ok=True)

In [3]:
def data_categorty_title(data_path,json_path):
    data = pd.read_csv(data_path,usecols = ['video_id','trending_date','category_id','publish_time','views','likes',
                                          'likes','dislikes','comment_count','comments_disabled','ratings_disabled',
                                          'video_error_or_removed'],encoding='latin-1')
    json = pd.read_json(json_path)
    
    dict_from_json = {}
    for row in json['items']:
        id = int(row['id'])
        title = row['snippet']['title']
        dict_from_json[id] = title
        
    category_title = pd.DataFrame(dict_from_json.values(),index = dict_from_json.keys(), columns = ['category_title'])
    data  = data.merge(category_title, how = 'inner', left_on='category_id',right_index=True)
    data.drop('category_id', inplace=True, axis=1)
    
    data = data[data['video_error_or_removed'] == False]
    data.drop('video_error_or_removed',inplace = True, axis = 1)


    
    return data

    

In [4]:
# data =  data_categorty_title('Data/USvideos.csv','Data/US_category_id.json')
# data

In [5]:
path = 'Data'
csv_files = glob.glob(path + "/*.csv")
json_files = glob.glob(path + "/*.json")
datas = {}

# print(len(csv_files))
for i in range(len(csv_files)):
    data = data_categorty_title(csv_files[i],json_files[i])
    data['Country'] = csv_files[i][5:7]
    datas[i] = data




    


In [6]:
final_data = pd.concat(datas.values(), ignore_index=True)
final_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372952 entries, 0 to 372951
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           372952 non-null  object
 1   trending_date      372952 non-null  object
 2   publish_time       372952 non-null  object
 3   views              372952 non-null  int64 
 4   likes              372952 non-null  int64 
 5   dislikes           372952 non-null  int64 
 6   comment_count      372952 non-null  int64 
 7   comments_disabled  372952 non-null  bool  
 8   ratings_disabled   372952 non-null  bool  
 9   category_title     372952 non-null  object
 10  Country            372952 non-null  object
dtypes: bool(2), int64(4), object(5)
memory usage: 26.3+ MB


In [7]:
final_data_without_duplicates = final_data.drop_duplicates(subset=['video_id'],keep = 'last')
final_data_without_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182265 entries, 10 to 372951
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   video_id           182265 non-null  object
 1   trending_date      182265 non-null  object
 2   publish_time       182265 non-null  object
 3   views              182265 non-null  int64 
 4   likes              182265 non-null  int64 
 5   dislikes           182265 non-null  int64 
 6   comment_count      182265 non-null  int64 
 7   comments_disabled  182265 non-null  bool  
 8   ratings_disabled   182265 non-null  bool  
 9   category_title     182265 non-null  object
 10  Country            182265 non-null  object
dtypes: bool(2), int64(4), object(5)
memory usage: 14.3+ MB


In [8]:
from sklearn.model_selection import train_test_split

_, final_data_without_duplicates_10 = train_test_split(final_data_without_duplicates,test_size=.1,random_state=5)

In [9]:
final_data_without_duplicates_10_mahmoud,final_data_without_duplicates_10_elbagory = train_test_split(final_data_without_duplicates_10,test_size=.5,random_state=10)


In [10]:
final_data_without_duplicates_10_mahmoud

,video_id,trending_date,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category_title,Country
100313,BIiVSBiHKPo,18.30.04,2018-04-29T05:03:59.000Z,779723,1989,190,962,False,False,Sports,FR
301844,VsWzjqAyIvY,18.11.05,2018-05-10T05:00:03.000Z,47327,6310,47,448,False,False,People & Blogs,RU
70708,4EHWpm1Aq_Q,18.23.04,2018-04-22T17:46:42.000Z,3414,205,17,196,False,False,Sports,DE
133038,kGOmPmILndU,17.15.11,2017-11-07T13:30:04.000Z,1500067,65867,1836,5638,False,False,Music,GB
303435,KlJbIxWJsRA,17.23.11,2017-11-22T16:23:05.000Z,271944,18582,1308,2257,False,False,Entertainment,RU
...,...,...,...,...,...,...,...,...,...,...,...
306191,ipDnxapmtSE,18.05.03,2018-03-05T01:55:26.000Z,318092,3674,52,412,False,False,Entertainment,RU
43206,g1-W7FOzItY,17.17.12,2017-12-16T11:00:04.000Z,41036,3509,185,201,False,False,Entertainment,DE
334099,9DUfFHaUpxg,18.25.03,2018-03-20T14:24:31.000Z,1732872,46176,513,2662,False,False,People & Blogs,US
287399,Zi3Ig5DaWYw,18.12.03,2018-03-11T05:31:30.000Z,70105,203,61,0,False,False,Sports,MX


In [12]:
final_data.to_csv('Data/final_data/final_data.csv')


In [11]:
final_data_without_duplicates_10_elbagory.to_csv('Data/final_data/final_data_without_duplicates_10_elbagory.csv')
final_data_without_duplicates_10_mahmoud.to_csv('Data/final_data/final_data_without_duplicates_10_mahmoud.csv')
final_data_without_duplicates_10.to_csv('Data/final_data/final_data_without_duplicates_10.csv')
final_data_without_duplicates.to_csv('Data/final_data/final_data_without_duplicates.csv')



In [29]:
final_data_without_duplicates_10_elbagory.to_csv?

In [31]:
df = pd.read_csv('Data/final_data/final_data_without_duplicates_10_elbagory.csv',index_col=0)
df.head()

,video_id,trending_date,publish_time,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,category_title,Country
103063,azcXcIQWucc,18.19.01,2018-01-17T15:10:54.000Z,53717,250,90,159,False,False,People & Blogs,FR
327950,fkQsexlUWtA,17.24.11,2017-11-23T12:40:22.000Z,114410,10140,140,1434,False,False,Howto & Style,RU
208481,I_uCGbxW6jA,18.18.03,2018-03-17T10:00:04.000Z,141300,4090,66,405,False,False,Entertainment,JP
299094,DRMWv7qMwyM,18.11.03,2018-03-10T17:40:40.000Z,25503,186,35,108,False,False,People & Blogs,RU
43727,0y8SjibXed0,17.24.12,2017-12-23T11:37:53.000Z,9427,23,46,0,False,False,Entertainment,DE


1. What are trending videos over all data --> Top 10 --> Category

2. What are trending videos over all data --> Top 10 --> Category --> Each Country

3. What are trending videos over all data in Weekends --> Top 10

4. Time Series --> All data --> Category

5. Length of video --> Web scraping

6. NLP & Sentiment Analysis --> Title